# Реализация tgnews

In [ ]:
# Re-install pytables for hdf support (google colab has old & glitchy pytables)
#!pip uninstall -y tables
#!pip install tables

In [1]:
import pandas as pd
df = pd.read_hdf('fe1.h5')
df2 = pd.read_hdf('fe2.h5')
df[df2.columns] = df2
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 227654 entries, 0 to 467481
Data columns (total 39 columns):
filename         227654 non-null object
text             227654 non-null object
lang             227654 non-null object
url              103093 non-null object
site             103093 non-null object
time             103093 non-null object
title            103093 non-null object
description      102861 non-null object
news             103093 non-null object
politics         103093 non-null object
elections        103093 non-null object
legislation      103093 non-null object
incidents        103093 non-null object
crime            103093 non-null object
society          103093 non-null object
markets          103093 non-null object
finance          103093 non-null object
business         103093 non-null object
economy          103093 non-null object
gadgets          103093 non-null object
auto             103093 non-null object
apps             103093 non-null object
internet 

In [2]:
df.head()

,filename,text,lang,url,site,time,title,description,news,politics,...,music,games,books,arts,entertainment,health,biology,physics,genetics,science
0,dataset/20191108/15/1491978796208217930.html,\n\n\n\n\n\n\n\n\n\n\n\nNewcastle United v Bou...,en,https://www.theguardian.com/football/2019/nov/...,the Guardian,2019-11-08T15:14:57+00:00,Newcastle United v Bournemouth: match preview,Newcastle finally found attacking rhythm last ...,False,False,...,False,False,False,False,False,False,False,False,False,False
4,dataset/20191108/15/6359253557838878602.html,\n\n\n\n\n\n\n\n\n\n\n\nМинкомсвязь негативно ...,ru,https://www.destructoid.com/american-truck-sim...,Destructoid,2019-11-08T15:00:00+00:00,American Truck Simulator is free for the weekend,"Between Red Dead Redemption 2, Death Stranding...",False,False,...,False,False,False,False,False,False,False,False,False,False
6,dataset/20191108/15/8549607152500932485.html,\n\n\n\n\n\n\n\n\n\n\n\nTether Responds to Mar...,en,https://www.channel3000.com/news/national-news...,CHANNEL3000,2019-11-08T15:40:38+00:00,GM sells Ohio plant to electric pickup truck s...,"A massive Lordstown, Ohio, auto assembly plant...",True,False,...,False,False,False,False,False,False,False,False,False,False
7,dataset/20191108/15/3752801377276103486.html,\n\n\n\n\n\n\n\n\n\n\n\nAnger over Las Vegas b...,en,https://www.reuters.com/article/us-france-dohe...,Reuters,2019-11-08T15:37:01+00:00,Paris police detain rocker Pete Doherty for bu...,PARIS (Reuters) - British rocker Pete Doherty ...,False,False,...,False,False,False,False,False,False,False,False,False,False
10,dataset/20191108/15/5545714965965021212.html,\n\n\n\n\n\n\n\n\n\n\n\nAmerican Truck Simulat...,en,https://www.theedgemarkets.com/article/rakyat-...,The Edge Markets,2019-11-08T15:33:46+00:00,Rakyat will understand if told the true pictur...,PONTIAN (Nov 8): The Tanjung Piai by-election ...,False,False,...,False,False,False,False,False,False,False,False,False,False


## tgnews languages

In [9]:
df['lang'].value_counts()

ru    120468
en    107186
Name: lang, dtype: int64

## Обучение

In [18]:
top_cat = 'News'
cats = {
    'Society': ('Politics', 'Elections', 'Legislation', 'Incidents', 'Crime'),
    'Economy': ('Markets', 'Finance', 'Business'),
    'Technology': ('Gadgets', 'Auto', 'Apps', 'Internet'),
    'Sports': ('Sport', 'E-Sport', 'ESport'),
    'Entertainment': ('Movies', 'Music', 'Games', 'Books', 'Arts'),
    'Science': ('Health', 'Biology', 'Physics', 'Genetics'),
}
other_cat = 'Other'
rev_cats = dict()
for cat in cats:
  for c in cats[cat]:
    rev_cats[c] = cat
  rev_cats[cat] = cat
all_cats = [top_cat] + list(rev_cats.keys())
def get_cat_filters(df, lang):
    global rev_cats, top_cat, other_cat
    cat_filters = {cat: df[cat.lower()] == True for cat in set(rev_cats.values())}
    for cat in rev_cats:
        cat_filters[rev_cats[cat]] = cat_filters[rev_cats[cat]] | (df[cat.lower()] == True)
    for cat in cat_filters:
        cat_filters[cat] = cat_filters[cat] & (df[top_cat.lower()] == True)
    cat_filters[top_cat] = df[top_cat.lower()] == True
    cat_filters[other_cat] = df[top_cat.lower()] == True
    for cat in rev_cats:
        cat_filters[other_cat] = cat_filters[other_cat] & (df[cat.lower()] == False)
    for cat in cat_filters:
        cat_filters[cat] = cat_filters[cat] & (df['lang'] == lang)
    return cat_filters

In [6]:
import nltk
#nltk.download('stopwords', download_dir='nltk_data')
nltk.data.path.append('nltk_data')
from nltk.corpus import stopwords

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
def get_model(lang):
    long_lang = 'english' if lang == 'en' else 'russian'
    pl = Pipeline([
        ('cv', CountVectorizer(stop_words=stopwords.words(long_lang))),
        ('lr', LogisticRegression(verbose=3)),
    ], verbose=3)
    return pl

In [22]:
import pickle
def create_model(cat, lang):
    m = get_model(lang)
    df_lang = df[df['lang'] == lang]
    m.fit(df_lang['text'].values, get_cat_filters(df_lang, lang)[cat].astype('int').values)
    with open('%s_%s.pkl' % (cat,lang), 'wb') as f:
        pickle.dump(m, f)

In [33]:
from joblib import Parallel, delayed
Parallel(n_jobs=2, verbose=3)(delayed(create_model)(cat, lang)
                               for lang in ['en', 'ru'] for cat in [top_cat] + list(cats.keys()))
for lang in ['en', 'ru']:
    for cat in [top_cat] + list(cats.keys()):
        !ls -l {cat}_{lang}.pkl

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


-rw-r--r-- 1 root root 19357399 Nov 23 12:08 News_en.pkl


[Parallel(n_jobs=2)]: Done  14 out of  14 | elapsed: 14.3min finished


-rw-r--r-- 1 root root 19357399 Nov 23 12:08 Society_en.pkl
-rw-r--r-- 1 root root 19357399 Nov 23 12:11 Economy_en.pkl
-rw-r--r-- 1 root root 19357399 Nov 23 12:10 Technology_en.pkl
-rw-r--r-- 1 root root 19357399 Nov 23 12:13 Sports_en.pkl
-rw-r--r-- 1 root root 19357399 Nov 23 12:13 Entertainment_en.pkl
-rw-r--r-- 1 root root 19357399 Nov 23 12:15 Science_en.pkl
-rw-r--r-- 1 root root 38227851 Nov 23 12:15 News_ru.pkl
-rw-r--r-- 1 root root 38227851 Nov 23 12:17 Society_ru.pkl
-rw-r--r-- 1 root root 38227851 Nov 23 12:17 Economy_ru.pkl
-rw-r--r-- 1 root root 38227851 Nov 23 12:18 Technology_ru.pkl
-rw-r--r-- 1 root root 38227851 Nov 23 12:19 Sports_ru.pkl
-rw-r--r-- 1 root root 38227851 Nov 23 12:20 Entertainment_ru.pkl
-rw-r--r-- 1 root root 38227851 Nov 23 12:20 Science_ru.pkl


In [46]:
def categorize(cat, lang):
    global df
    df_lang = df[df['lang'] == lang]
    ret = [(lang, 'input', df_lang.shape[0])]
    with open('%s_%s.pkl' % (cat, lang), 'rb') as f:
        m = pickle.load(f)
        result = m.predict(df_lang['text'].values)
        ret += [(lang, cat, len(df_lang[result == 1]['filename'].values))]
    return ret

## tgnews news

In [50]:
%%time
for ret in Parallel(n_jobs=-1, verbose=3)(delayed(categorize)(top_cat, lang) for lang in ['en', 'ru']):
    for lang, cat, cnt in ret:
        print(lang, cat, cnt)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.


en input 107186
en News 11403
ru input 120468
ru News 18232
CPU times: user 2.11 s, sys: 2.04 s, total: 4.14 s
Wall time: 43 s


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:   42.7s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:   42.7s finished


## tgnews categories

In [54]:
%%time
for ret in Parallel(n_jobs=4, verbose=3)(delayed(categorize)(cat, lang) for lang in ['en', 'ru'] for cat in cats):
    for (lang, cat, cnt) in ret:
        print(lang, cat, cnt)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  12 | elapsed:  1.8min remaining:   21.6s


en input 107186
en Society 947
en input 107186
en Economy 639
en input 107186
en Technology 330
en input 107186
en Sports 1119
en input 107186
en Entertainment 497
en input 107186
en Science 244
ru input 120468
ru Society 1561
ru input 120468
ru Economy 944
ru input 120468
ru Technology 435
ru input 120468
ru Sports 1421
ru input 120468
ru Entertainment 571
ru input 120468
ru Science 257
CPU times: user 12 s, sys: 10.3 s, total: 22.3 s
Wall time: 1min 55s


[Parallel(n_jobs=4)]: Done  12 out of  12 | elapsed:  1.9min finished
